<a href="https://colab.research.google.com/github/showblue/ai-agent-foundation-labs/blob/main/Intro_to_LCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here's a very simple LCEL chain that does not use any LLM

Example: Convert text → Uppercase → Add exclamation



In [ ]:
from langchain_core.runnables import RunnableLambda

# Step 1: Make input uppercase
uppercase = RunnableLambda(lambda text: text.upper())

# Step 2: Add an exclamation mark
add_exclaim = RunnableLambda(lambda text: text + "!")

# LCEL pipeline using the pipe operator
chain = uppercase | add_exclaim

# Run the chain
result = chain.invoke("hello world LCEL")
print(result)


HELLO WORLD LCEL!


Minimal RAG Chain (no LLM needed)

In [ ]:
from langchain_core.runnables import RunnableLambda

# A tiny in-memory "vector store" 😄
docs = [
    "LangChain helps developers build LLM applications.",
    "LCEL is a pipe-style syntax for chaining components."
]

# A fake retrieval function: return docs containing the keyword
def retrieve(query: str):
    return [d for d in docs if query.lower() in d.lower()]

# A fake answer function: stitch context into a response
def answer_with_context(context_docs):
    if not context_docs:
        return "No information found."
    return "Here is what I found:\n" + "\n".join(context_docs)

# Components wrapped as LCEL runnables
retriever = RunnableLambda(retrieve)
answerer = RunnableLambda(answer_with_context)

# RAG pipeline using LCEL
rag_chain = retriever | answerer

# Run it
result = rag_chain.invoke("LangChain")
print(result)


Here is what I found:
LangChain helps developers build LLM applications.


In [ ]:
## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich arxiv pymupdf faiss-cpu
# %pip install -U langchain-community


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.4.1 requires langchain-core<2.0.0,>=1.0.1, but you have langchain-core 0.3.80 which is incompatible.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.11 which is incompatible.


In [ ]:
import os
from google.colab import userdata

# os.environ["NVIDIA_API_KEY"] = "nvapi-..."
os.environ["NVIDIA_API_KEY"] = userdata.get("NVIDIA_API_KEY")

## If you encounter a typing-extensions issue, restart your runtime and try again
from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

import warnings
warnings.filterwarnings('ignore')


In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

# Our mini knowledge base
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]
## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

result = retriever.invoke("Where are the Rocky Mountains?")

pprint(result)

[
    Document(
        id='f47574bd-d9bd-4a82-bed0-fd9a89056a08',
        metadata={},
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        id='0da32cc0-07e2-4155-a9d1-11188eb3a8b1',
        metadata={},
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    ),
    Document(
        id='86ce09df-2d68-4866-b917-a89b4d680768',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='0aaea3a8-801e-4cc0-a64b-6e97fbcbd6c8',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]